In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd

df = pd.read_excel("../graphm_messages.xlsx",  converters = {'message_reply_id': str, "message_id": str, "sender_team_id":str, "replier_team_id": str})
df = df.drop(columns=[c for c in df.columns if "Unnamed" in c])

In [ ]:
from keywords import KeywordExtractor
extractor = KeywordExtractor()

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
df["generated_topics"] = df["message_body"].map( lambda text:"\n".join([word for word, score in extractor.extract(text)]))

In [ ]:
values_all = list(map(lambda x: x.split("\n"), list(df["generated_topics"].values)))
#values_questions_only = list(map(lambda x: x.split("\n"), list(df[df["message_type"] == "QUESTION"]["generated_topics"].values)))
all_keywords = list(sum(values_all, []))

In [ ]:
topics = extractor.cluster_keywords(all_keywords, n=12)
print(set(topics))

In [ ]:
lookup = dict(set(zip(all_keywords, topics)))

def map_topics(x):
    keywords = x.split("\n")
    mapped = set(map(lambda y: lookup[y], keywords))
    return "\n".join(mapped)


In [ ]:
df["clustered_topics"] = df["generated_topics"].map(map_topics)

In [ ]:
dfs = [x for _, x in df.groupby('message_id')]

In [ ]:
def process_qa(df):
    question = df[df["message_type"] == "QUESTION"]["message_body"].values[0]
    answers = df[df["message_type"] == "ANSWER"]["message_body"]
    ranks = extractor.ranker(question, answers)

    scores = [s for i,t,s in ranks]
    df["relevance"] = [0] + scores
    return df

In [ ]:
import pandas as pd
final_df = pd.concat([process_qa(df) for df in dfs])

In [ ]:
final_df.to_excel("processed.xlsx")

In [ ]:
from classifier import MessageClassifer
message_classifer = MessageClassifer(MODEL_NAME="intent-model_1/intent-model_1")

In [ ]:
text = final_df["message_body"].values[1]
processed = message_classifer.text_to_sents(text)

for s in processed:
    r = message_classifer.predict(s)
    print(s, r)

